# Playground

A safe space for experimentation.

In [78]:
%load_ext autoreload
%autoreload 2

import base64
import dill
import requests
from datetime import datetime, timedelta
from etl_db_manager import ETLDataBaseManager
from etl_task import ETLTask, ETLTaskFunction
from utility import base64encode_obj, base64decode_obj

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
def load_data_airnoise(sensor_id:str):
    """ Fetches air/noise data from the API. 
        @param sensor_id: Serial number of the sensor.
        @return: Data from API or [].
    """
    time_end = datetime.now()
    time_start = time_end - timedelta(seconds=15*60)
    data = []
    try:
        res = requests.post(f"https://data.smartdublin.ie/sonitus-api/api/data", json={ 
            'username': "dublincityapi",
            'password': "Xpa5vAQ9ki",
            'monitor': sensor_id,
            'start': time_start.timestamp(),
            'end': time_end.timestamp()
        })
        data = res.json()
    except Exception as e:
          raise print(f'Failed to load air noise data from source: {e}')
    return data

In [121]:
# Toy data load and save functions.
def load_data(src): 
    print(f'Loaded data from {src}.')

def save_data(dst): 
    print(f'Saved data to {dst}.')

In [171]:
def make_post_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.post(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_get_request(url, data={}):
    """
    Makes a post request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.get(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_delete_request(url, data={}):
    """
    Makes a delete request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.delete(url, params=data)
    return {'status': response.status_code, 'message': response.text}

def make_put_request(url, task_name, new_values):
    """
    Makes a put request.
    @param url: URL to post to.
    @data: Request body data.
    @return: Response.
    """
    response = requests.put(
        url=url, 
        params={"task_name": task_name}, 
        json=new_values
    )
    return {'status': response.status_code, 'message': response.json()}

In [214]:
db_man = ETLDataBaseManager(db_name="db_etl", db_path=".")

Database './db_etl.db' available :) .


In [206]:
db_man.query(q="SELECT * FROM etl_tasks WHERE status='scheduled'", params=[])

In [213]:
db_man.load_tasks()

[DEBUG] q = SELECT * FROM etl_tasks


In [188]:
delta_sec = timedelta(seconds=5)

In [216]:
# Create a new task.
task = ETLTask(
    name='task1',
    fun_data_load=ETLTaskFunction(function=load_data, args=['test_api']),
    fun_data_save=ETLTaskFunction(function=save_data, args=['test_db']),
    repeat_time_unit='seconds',
    repeat_interval=2
)
task_str = base64encode_obj(task)
make_post_request(url="http://127.0.0.1:8003/task", data={"task_str": task_str})

{'status': 200,
 'message': '{"status":200,"message":"Success. Task created and scheduled task1.","data":[]}'}

In [223]:
task = ETLTask(
    name='task2',
    fun_data_load=ETLTaskFunction(function=load_data, args=['task2']),
    fun_data_save=ETLTaskFunction(function=save_data, args=['task2']),
    repeat_time_unit='seconds',
    repeat_interval=2
)
task_str = base64encode_obj(task)
make_post_request(url="http://127.0.0.1:8003/task", data={"task_str": task_str})

{'status': 200,
 'message': '{"status":200,"message":"Success. Task created and scheduled task2.","data":[]}'}

In [215]:
# Delete a task.
make_delete_request(url="http://127.0.0.1:8003/task", data={'task_name': 'task1'})

{'status': 200,
 'message': '{"status":200,"message":"Success. Deleted task task1.","data":[]}'}

In [224]:
# Start scheduler.
make_get_request(url="http://127.0.0.1:8003/start_scheduler")

{'status': 200,
 'message': '{"status":200,"message":"Scheduler started.","data":[]}'}

In [225]:
# Stop scheduler.
make_get_request(url="http://127.0.0.1:8003/stop_scheduler")

{'status': 200,
 'message': '{"status":200,"message":"Scheduler stopped.","data":[]}'}

In [202]:
make_put_request(
    url="http://127.0.0.1:8003/task", 
    task_name="task1",
    new_values={
        'status': 'scheduled',
        "num_runs": 0
    }
)

{'status': 200,
 'message': {'status': 200,
  'message': "Success. Status of task task1 updated with new values {'status': 'scheduled', 'num_runs': 0}.",
  'data': []}}

In [167]:
requests.put(url="http://127.0.0.1:8003/task", 
    params={"task_name": "task1"}, 
    json={
        'status': 'running',
        'time_run_last_start': datetime.now().isoformat(),
        'time_run_last_end': datetime.now().isoformat(),
        "num_runs": 200
    }
).json()

{'status': 200,
 'message': "Success. Status of task task1 updated with new values {'status': 'running', 'time_run_last_start': '2024-03-16T00:19:29.088353', 'time_run_last_end': '2024-03-16T00:19:29.088353', 'num_runs': 200}.",
 'data': []}